In [6]:
import pandas as pd
import numpy as np
import pyperclip
import io
from typing import List, Dict, Tuple

In [7]:
def copy_df_column_to_clipboard(df, column_name):
    """
    Copies the specified column from a pandas DataFrame to the clipboard using pyperclip.

    Args:
        df (pd.DataFrame): The pandas DataFrame containing the data.
        column_name (str): The name of the column to copy.

    Returns:
        bool: True if the data was copied successfully, False otherwise.
    """
    try:
        # Check if the column exists in the DataFrame
        if column_name not in df.columns:
            print(f"Error: Column '{column_name}' not found in DataFrame.")
            return False

        # # Select the column and convert it to a string format
        # column_data = df[column_name].astype(str)
        # column_string = column_data.to_string(header=True, index=False)

        # Select the column
        column_data = df[column_name]

        # Convert to string with tab separation
        output = io.StringIO()
        column_data.to_csv(output, sep='\t', header=False, index=False)
        column_string = output.getvalue()
        output.close()

        # Copy the string to the clipboard using pyperclip
        pyperclip.copy(column_string)
        return True
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

In [8]:
class ProductionOrderScheduler:
    def __init__(self, excel_file_path: str, sheet_name: str = "P100"):
        """
        Initialize the scheduler with production data
        """
        self.df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
        self.scheduled_orders = []
        
    def is_triple_glazed(self, glass_type: str) -> bool:
        """Check if glass type is triple-glazed (9 or 9C)"""
        return glass_type in ['9', '9C']
    
    def is_urgent_order(self, goods_receiver: str) -> bool:
        """Check if order is urgent (ends with /C)"""
        return str(goods_receiver).endswith('/C')
    
    def is_small_order(self, quantity: int) -> bool:
        """Check if order is small (quantity <= 3)"""
        return quantity <= 3
    
    def get_special_sap_orders(self) -> Tuple[List[int], List[int]]:
        """Get orders with SAP numbers 808965 and 808966"""
        sap_808965 = self.df[self.df['sap_nr'] == 808965]['record_number'].tolist()
        sap_808966 = self.df[self.df['sap_nr'] == 808966]['record_number'].tolist()
        return sap_808965, sap_808966
    
    def separate_by_glass_type(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Separate orders by glass type (triple-glazed vs others)"""
        triple_glazed = self.df[self.df['glass_type'].apply(self.is_triple_glazed)]
        other_glazed = self.df[~self.df['glass_type'].apply(self.is_triple_glazed)]
        return triple_glazed, other_glazed
    
    def group_by_profile_color(self, df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """Group orders by profile color"""
        colors = {}
        for color in df['profile_color'].unique():
            colors[color] = df[df['profile_color'] == color].copy()
        return colors
    
    def sort_by_width(self, df: pd.DataFrame) -> pd.DataFrame:
        """Sort orders by width for better linking"""
        return df.sort_values('width')
    
    def check_urgent_position_constraint(self, orders: List[int], total_quantity: int) -> bool:
        """Check if urgent orders are positioned correctly"""
        cumulative_qty = 0
        for i, record_num in enumerate(orders):
            row = self.df[self.df['record_number'] == record_num].iloc[0]
            cumulative_qty += row['quantity']
            
            # Check Polska/C and Węgry/C constraint (first half)
            if row['goods_receiver'] in ['2101/Polska/C', '3301/Węgry/C']:
                if cumulative_qty > total_quantity / 2:
                    return False
            
            # Check Francja/C and Czechy/C constraint (first 80%)
            if row['goods_receiver'] in ['1201/Francja/C', '3701/Czechy/C']:
                if cumulative_qty > total_quantity * 0.8:
                    return False
        
        return True
    
    def schedule_sequence(self, df: pd.DataFrame, is_first_third: bool = False) -> List[int]:
        """Schedule a sequence of orders following the rules"""
        if df.empty:
            return []
        
        scheduled = []
        remaining = df.copy()
        
        # Group by profile color
        color_groups = self.group_by_profile_color(remaining)
        
        # Process white windows first, then colored ones with separators
        color_order = ['W']  # Start with white
        non_white_colors = [c for c in color_groups.keys() if c != 'W']
        
        small_order_count = 0
        variant_pro_exl_count = 0
        current_window_type = None
        
        for color in color_order + non_white_colors:
            if color not in color_groups or color_groups[color].empty:
                continue
                
            color_df = self.sort_by_width(color_groups[color])
            
            for _, row in color_df.iterrows():
                # Rule 13: No more than 3 small orders in sequence
                if self.is_small_order(row['quantity']):
                    if small_order_count >= 3:
                        continue
                    small_order_count += 1
                else:
                    small_order_count = 0
                
                # Rule 12: No more than 18 pcs of PRO and EXL in sequence
                if row['variant'] in ['PRO', 'EXL']:
                    if variant_pro_exl_count + row['quantity'] > 18:
                        continue
                    variant_pro_exl_count += row['quantity']
                else:
                    variant_pro_exl_count = 0
                
                # Rule 11: Keep R3 windows in sequences
                if current_window_type is None:
                    current_window_type = row['window_type']
                elif current_window_type == 'R3' and row['window_type'] != 'R3':
                    # Try to continue R3 sequence if possible
                    continue
                
                scheduled.append(row['record_number'])
            
            # Add white separators between colored groups (Rule 8)
            if color != 'W' and len(non_white_colors) > 1:
                white_separators = color_groups.get('W', pd.DataFrame())
                if not white_separators.empty:
                    # Add a few white windows as separators
                    separator_count = min(2, len(white_separators))
                    for i in range(separator_count):
                        if not white_separators.empty:
                            separator_row = white_separators.iloc[0]
                            scheduled.append(separator_row['record_number'])
                            white_separators = white_separators.iloc[1:]
        
        return scheduled
    
    def schedule_orders(self) -> List[int]:
        """Main scheduling function following all rules"""
        
        # Rule 1: Get special SAP orders for start and end
        sap_808965_orders, sap_808966_orders = self.get_special_sap_orders()
        
        # Choose start and end orders
        start_order = None
        end_order = None
        
        if sap_808965_orders:
            start_order = sap_808965_orders[0]
        if sap_808966_orders:
            end_order = sap_808966_orders[0]
        elif len(sap_808965_orders) > 1:
            end_order = sap_808965_orders[1]
        
        # Remove start and end orders from main scheduling
        remaining_df = self.df.copy()
        if start_order:
            remaining_df = remaining_df[remaining_df['record_number'] != start_order]
        if end_order:
            remaining_df = remaining_df[remaining_df['record_number'] != end_order]
        
        # Separate by glass type
        triple_glazed, other_glazed = self.separate_by_glass_type()
        
        # Remove start/end orders from glass type groups too
        if start_order:
            triple_glazed = triple_glazed[triple_glazed['record_number'] != start_order]
            other_glazed = other_glazed[other_glazed['record_number'] != start_order]
        if end_order:
            triple_glazed = triple_glazed[triple_glazed['record_number'] != end_order]
            other_glazed = other_glazed[other_glazed['record_number'] != end_order]
        
        # Rule 6: Split triple-glazed orders into two similar-sized parts
        triple_glazed_records = triple_glazed['record_number'].tolist()
        mid_point = len(triple_glazed_records) // 2
        
        first_triple = triple_glazed[triple_glazed['record_number'].isin(triple_glazed_records[:mid_point])]
        third_triple = triple_glazed[triple_glazed['record_number'].isin(triple_glazed_records[mid_point:])]
        
        # Schedule each part
        scheduled = []
        
        # Add start order
        if start_order:
            scheduled.append(start_order)
        
        # First part: Triple-glazed
        first_part = self.schedule_sequence(first_triple, is_first_third=True)
        scheduled.extend(first_part)
        
        # Second part: Other glazed
        second_part = self.schedule_sequence(other_glazed)
        scheduled.extend(second_part)
        
        # Third part: Triple-glazed
        third_part = self.schedule_sequence(third_triple)
        scheduled.extend(third_part)
        
        # Add end order
        if end_order:
            scheduled.append(end_order)
        
        # Add any remaining orders that weren't scheduled
        all_records = set(self.df['record_number'].tolist())
        scheduled_records = set(scheduled)
        remaining_records = all_records - scheduled_records
        
        scheduled.extend(list(remaining_records))
        
        return scheduled

def main():
    """
    Main function to run the production scheduler
    """
    # Initialize scheduler with the Excel file
    # Note: Replace 'production_plan.xlsx' with your actual file path
    scheduler = ProductionOrderScheduler('01_P100_AI_3.xlsx', 'P100')
    
    # Schedule the orders
    scheduled_order = scheduler.schedule_orders()
    
    # Create output DataFrame
    output_df = pd.DataFrame({'record_number': scheduled_order})
    
    # Save to Excel file
    output_df.to_excel('scheduled_production_order.xlsx', index=False)
    
    # Print the result
    print("Scheduled Production Order (Record Numbers):")
    for record_num in scheduled_order:
        print(record_num)
    
    return output_df

# if __name__ == "__main__":
#     main()

In [9]:
output_df = main()
copy_df_column_to_clipboard(output_df, 'record_number')

Scheduled Production Order (Record Numbers):
6
3
7
15
13
18
25
27
26
32
33
34
39
43
40
19
22
24
31
21
38
14
12
5
42
41
1
17
2
16
9
28
36
29
35
44
8
45
23
30
37
20
10
11
4
46


True